# 1. 데이터 로드

In [2]:
import os
import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [3]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [4]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [5]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [6]:
srch_keyword

['keyword_final']

In [7]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))

In [8]:
print(collected_keywords_data)

         연관키워드  월간검색수_합계 검색어
0           주식  528900.0  주식
1          미주부     390.0  주식
2        김현준대표     960.0  주식
3         퀀트투자    6710.0  주식
4         주식투자   12390.0  주식
...        ...       ...  ..
30599   수원유언공증      18.0  증여
30600   법인주소변경    1960.0  증여
30601   인터넷법무사     470.0  증여
30602    환지예정지     340.0  증여
30603  지정유언집행자      90.0  증여

[30604 rows x 3 columns]


In [9]:
sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)

# 각 '검색어'별로 분리된 DataFrame을 리스트에 저장
grouped = sorted_df.groupby('검색어')
df_list = [group for _, group in grouped]
print(len(df_list))

45


C:\Users\chohy\AppData\Local\Temp\ipykernel_26828\3376418344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sorted_df = collected_keywords_data.groupby('검색어').apply(lambda x: x.sort_values('월간검색수_합계', ascending=False)).reset_index(drop=True)


In [10]:
from utils import merge_and_mark_duplicates_limited
collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data



,연관키워드,월간검색수_합계,검색어,중복검색어
0,달러환율,2042300.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식"
1,적금계산기,642400.0,CMA금리비교,"CMA금리비교,CMA통장"
2,다우지수,561800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주"
3,적금,468200.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식"
4,코스닥,444900.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매..."
...,...,...,...,...
1139,S&P500지수,33500.0,테마주,"테마주,퇴직연금"
1140,TQQQ,32690.0,테마주,테마주
1141,코스피야간선물,31180.0,테마주,테마주
1142,양자컴퓨터관련주,30470.0,테마주,테마주


In [11]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
new_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성']

for column in new_columns:
    collected_keywords_data[column] = 0
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw,일별급상승,주별급상승,월별급상승,주별지속상승,월별지속상승,월별규칙성
0,달러환율,2042300.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
1,적금계산기,642400.0,CMA금리비교,"CMA금리비교,CMA통장",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
2,다우지수,561800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
3,적금,468200.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
4,코스닥,444900.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1139,S&P500지수,33500.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1140,TQQQ,32690.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1141,코스피야간선물,31180.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1142,양자컴퓨터관련주,30470.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0


In [12]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [13]:
import asyncio
import trend  # 가정: trend 모듈에 trend_maincode 함수가 정의되어 있음

# 비동기 메인 함수 수정
async def trend_main(df, clients):
    # 파라미터 설정
    params = {
        "search_keywords": list(df['연관키워드']),
        "id": df['id'].iloc[0],
        "pw": df['pw'].iloc[0],
        "api_url": "https://openapi.naver.com/v1/datalab/search",
        "name": '연관검색어'
    }
    api_url = "https://openapi.naver.com/v1/datalab/search"
    
    # trend_maincode 함수 실행
    results = await trend.trend_maincode(params, clients, api_url)
    return results

async def run_all(df_list, clients):
    tasks = [trend_main(df, clients) for df in df_list]
    results = await asyncio.gather(*tasks)
    return results

clients = get_secret("clients")  # clients 정보를 로드
# 이벤트 루프 실행

results = asyncio.run(run_all(df_list, clients))

In [14]:
from select_keyword import select_keyword, rising_keyword_analysis, monthly_rule

start_time = time.time()
select_periods = ['daily', 'weekly', 'month']
rising_periods=['weekly', 'month']

formatted_today, today_date = utils.get_today_date()
month_rule_list=[]
select_list=[[],[],[]]

rising_list=[[],[]]
# 월별, 주별, 일별 키워드 분석 실행

    # 각 분석 기간에 대해 결과 집합을 순회합니다.
for keyword_group in results:
    # 키워드 그룹의 각 키워드 데이터프레임에 대해 순회합니다.
    for keyword_data in keyword_group:
        # 월별 규칙을 적용하여 결과를 가져옵니다.
        monthly_data, monthly_chart, similarity_rate, rising_months = monthly_rule(keyword_data, today_date, 'month')
        if monthly_data is not None:
            # 결과 데이터프레임의 열 이름을 가져옵니다.
            column_names = monthly_data.columns
            
            # 결과 데이터프레임에서 값 리스트를 추출합니다.
            data_values_list = monthly_data[column_names].values
            # 월별 차트에 데이터 값을 추가합니다.
            monthly_chart['Indicator'] = data_values_list
            monthly_chart['InfoData'] = similarity_rate
            # 상승 월 정보를 추가합니다. 상승 월이 없는 경우 0으로 설정합니다.
            monthly_chart['RisingMonth'] = rising_months[0] if len(rising_months) > 0 else 0
            
            # 최종 결과 리스트에 수정된 월별 차트를 추가합니다.
            month_rule_list.append(monthly_chart)
                
# 주별, 월별 상승 키워드 분석 실행
rising_analysis_periods = ['weekly', 'month']
i=0
for period in rising_analysis_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            rising_tmp, rising_graph, rising_info = rising_keyword_analysis(keyword_df, today_date, period)
            if rising_tmp is not None:
                column_names=rising_tmp.columns
                data_values_list = rising_tmp[column_names].values
                rising_graph['Indicator'] = data_values_list
                rising_graph['InfoData'] = rising_info

                rising_list[i].append(rising_graph)
    i=i+1

    


i = 0
# 일별, 주별, 월별 키워드 선택 실행
for period in select_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            selected_tmp, selected_graph, selected_info = select_keyword(keyword_df, today_date, period)
            if selected_graph is not None:
                # 데이터프레임의 열 이름을 출력합니다.
                selected_graph['InfoData'] = selected_info
                select_list[i].append(selected_graph)
            else:
                pass
    i += 1

end_time = time.time()
print(f"Analysis completed in {end_time - start_time} seconds.")

월별 규칙성 키워드 발견 : 소득공제
월별 규칙성 키워드 발견 : 체크카드혜택
월별 규칙성 키워드 발견 : 연말정산
월별 규칙성 키워드 발견 : 샘플
월별 규칙성 키워드 발견 : 목업
월별 규칙성 키워드 발견 : 연말정산하는법
월별 규칙성 키워드 발견 : AIG손해보험
월별 규칙성 키워드 발견 : INSURANCE
월별 규칙성 키워드 발견 : 보험설계사
월별 규칙성 키워드 발견 : 건강보험종류
월별 규칙성 키워드 발견 : AIG
월별 규칙성 키워드 발견 : 18K금시세
월별 규칙성 키워드 발견 : 금시세18K
월별 규칙성 키워드 발견 : 알바
월별 규칙성 키워드 발견 : 일자리
월별 규칙성 키워드 발견 : 앱
월별 규칙성 키워드 발견 : 중고차어플
월별 규칙성 키워드 발견 : 동물그림
월별 규칙성 키워드 발견 : 석상
월별 규칙성 키워드 발견 : 명함제작
월별 규칙성 키워드 발견 : 주택관리사
월별 규칙성 키워드 발견 : 월세
월별 규칙성 키워드 발견 : 상가임대
월별 규칙성 키워드 발견 : 부동산매매
월별 규칙성 키워드 발견 : 상속세세율
월별 규칙성 키워드 발견 : 한정승인
월별 규칙성 키워드 발견 : 상속재산분할협의서
월별 규칙성 키워드 발견 : 상속포기신고서
월별 규칙성 키워드 발견 : 상속포기신청
월별 규칙성 키워드 발견 : 연말정산간소화
월별 규칙성 키워드 발견 : 원천징수
월별 규칙성 키워드 발견 : 부가세
월별 규칙성 키워드 발견 : 회계
월별 규칙성 키워드 발견 : 법무사
월별 규칙성 키워드 발견 : 차용증양식
월별 규칙성 키워드 발견 : 전세권설정
월별 규칙성 키워드 발견 : 등기
월별 규칙성 키워드 발견 : 아파트월세
월별 규칙성 키워드 발견 : 평생교육원
월별 규칙성 키워드 발견 : 해외선물
월별 규칙성 키워드 발견 : 보장성보험
월별 규칙성 키워드 발견 : 자동차보험
월별 규칙성 키워드 발견 : KB자동차보험
월별 규칙성 키워드 발견 : 케이비손해보험
월별 규칙성 키워드 발견 : 근로계약서
월별 규칙성 키워드 발견 : 소득공제용연금저축


In [15]:
def update_keywords_flag(dataframe, data_list, flag_name):
    unique_associated_keywords = set()
    for df in data_list:
        unique_associated_keywords.update(df['연관검색어'].unique())
    for index, row in dataframe.iterrows():
        if row['연관키워드'] in unique_associated_keywords:
            dataframe.at[index, flag_name] = 1

flags_and_lists = [
    ("일별급상승", select_list[0]),
    ("주별급상승", select_list[1]),
    ("월별급상승", select_list[2]),
    ("주별지속상승", rising_list[0]),
    ("월별지속상승", rising_list[1]),
    ("월별규칙성", month_rule_list),
]

for flag_name, data_list in flags_and_lists:
    update_keywords_flag(collected_keywords_data, data_list, flag_name)


In [16]:
collected_keywords_data

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw,일별급상승,주별급상승,월별급상승,주별지속상승,월별지속상승,월별규칙성
0,달러환율,2042300.0,CMA금리비교,"CMA금리비교,WTI,달러환율,미국금리,미국주식,주가지수,주식",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
1,적금계산기,642400.0,CMA금리비교,"CMA금리비교,CMA통장",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
2,다우지수,561800.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,급등주,달러환율,미국금리,미국주식,테마주",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
3,적금,468200.0,CMA금리비교,"CMA금리비교,CMA통장,금리,급등주,돈버는법,미국금리,배당주,재테크,주가지수,주식",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
4,코스닥,444900.0,CMA금리비교,"CMA금리비교,CMA통장,ETF,WTI,개인연금,달러환율,배당주,신규상장,외국인순매...",RRYYzqMgic6itMdMQNdP,6JjVKXRLfV,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1139,S&P500지수,33500.0,테마주,"테마주,퇴직연금",mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1140,TQQQ,32690.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1141,코스피야간선물,31180.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0
1142,양자컴퓨터관련주,30470.0,테마주,테마주,mZtQ5QquBFIrshEwfZBv,jAIZheOMh8,0,0,0,0,0,0


구글이나 네이버 merge하면 끝 근데 상승월이랑 활동성 컬럼 추가해야댐

In [17]:
import pandas as pd
def process_data(data, condition, type_label, data_lists):
    # 조건에 맞는 데이터 필터링
    filtered_data = data[data[condition] == 1].copy()
    filtered_data['유형'] = type_label
    
    # 불필요한 열 삭제
    columns_to_drop = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성', 'id', 'pw', '검색어']
    filtered_data.drop(columns_to_drop, axis=1, inplace=True)
    
    # 인덱스 재설정
    filtered_data.reset_index(drop=True, inplace=True)
    
    # '지표' 열 초기화
    filtered_data['지표'] = None  
    
    # '지표' 열에 데이터 채우기
    for i, df in enumerate(data_lists):
        if i < len(filtered_data):  
            filtered_data.at[i, '지표'] = str(df['InfoData'].iloc[0]) + '%' 
    # '상승월' 열 추가
    filtered_data['상승월'] = None
    
    return filtered_data

info_result_daily_select = process_data(collected_keywords_data, '일별급상승', '일별급상승', select_list[0])
info_result_weekly_select = process_data(collected_keywords_data, '주별급상승', '주별급상승', select_list[1])
info_result_monthly_select = process_data(collected_keywords_data, '월별급상승', '월별급상승', select_list[2]) 

info_result_weekly_continuous = process_data(collected_keywords_data, '주별지속상승', '주별지속상승', rising_list[0])

info_result_monthly_continuous = process_data(collected_keywords_data, '월별지속상승', '월별지속상승', rising_list[1])

info_result_monthly_pattern = process_data(collected_keywords_data, '월별규칙성', '월별규칙성', month_rule_list)

info_result_final = pd.concat([info_result_daily_select,info_result_weekly_select, info_result_monthly_select,\
                               info_result_weekly_continuous, info_result_monthly_continuous,\
                                  info_result_monthly_pattern]).reset_index(drop=True)
info_result_final

,연관키워드,월간검색수_합계,중복검색어,유형,지표,상승월
0,사망보험,5760.0,"건강보험,의료비보험",일별급상승,19.79%,None
1,삼성SDI주가,294000.0,"공모주,신규상장,주가지수,테마주,특징주",일별급상승,258.86%,None
2,NFT코인,5920.0,디지털자산,일별급상승,45.81%,None
3,주식복리계산기,16580.0,신탁,일별급상승,7.28%,None
4,GTXB노선,37410.0,아파트청약,일별급상승,403.75%,None
...,...,...,...,...,...,...
401,케이비손해보험,24790.0,자동차보험,월별규칙성,94.6%,None
402,근로계약서,40600.0,"적립식펀드,퇴직연금",월별규칙성,93.57%,None
403,소득공제용연금저축,15520.0,중개형ISA,월별규칙성,93.07%,None
404,직계존속,33800.0,증여,월별규칙성,90.91%,None


## Graph_result_df 

In [18]:
def select_and_process(df, condition, type_label):
    # 조건에 맞는 데이터 선택
    selected_df = df[df[condition] == 1].copy()
    selected_df['유형'] = type_label
    
    # 불필요한 열 삭제
    drop_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성', 'id', 'pw', '검색어', '월간검색수_합계', '중복검색어']
    selected_df.drop(drop_columns, axis=1, inplace=True)
    
    # 인덱스 초기화
    selected_df.reset_index(drop=True, inplace=True)
    
    return selected_df

# 각 조건에 대해 함수 호출
conditions_and_labels = [
    ('일별급상승', '일별급상승'),
    ('주별급상승', '주별급상승'),
    ('월별급상승', '월별급상승'),
    ('주별지속상승', '주별지속상승'),
    ('월별지속상승', '월별지속상승'),
    ('월별규칙성', '월별규칙성'),
]

# 빈 리스트를 생성하여 각 조건에 대한 결과를 저장
selected_dfs = []

for condition, label in conditions_and_labels:
    selected_df = select_and_process(collected_keywords_data, condition, label)
    selected_dfs.append(selected_df)

# 필요에 따라 selected_dfs 리스트에서 각 데이터프레임에 접근할 수 있습니다.
# 예: daily_select_df = selected_dfs[0]

concatenated_df = pd.concat(selected_dfs, ignore_index=True)


In [19]:
# 결과를 저장할 빈 리스트 초기화
matching_indices = []

# results 리스트를 반복하며 각 데이터프레임 탐색
for i, inner_list in enumerate(results):
    for j, df in enumerate(inner_list):
        asset_name = df.columns[0]  # 각 데이터프레임의 열 이름 (가상자산 이름)
        
        # concatenated_df에서 asset_name과 일치하는 연관키워드를 찾고, 해당 유형을 가져옴
        matching_rows = concatenated_df[concatenated_df['연관키워드'] == asset_name]
        
        # 일치하는 연관키워드가 있다면
        if not matching_rows.empty:
            for _, row in matching_rows.iterrows():
                type_name = row['유형']  # 일치하는 유형 이름
                # 결과 리스트에 인덱스와 유형을 추가
                matching_indices.append([i, j, type_name])


# 초기 df_result DataFrame을 생성합니다. 이는 나중에 각 결과를 쌓기 위한 빈 DataFrame입니다.
import pandas as pd
from datetime import datetime, timedelta

# matching_indices와 results가 정의되어 있다고 가정
graph_result = pd.DataFrame()

for idx in matching_indices:
    temp_df = results[idx[0]][idx[1]].reset_index()
    name = temp_df.columns[1]
    temp_df['연관검색어'] = name
    temp_df.rename(columns={'date': '검색일자', name: '검색량'}, inplace=True)
    temp_df['유형'] = idx[2]
    temp_df['오늘일자'] = datetime.today().strftime('%Y-%m-%d')
    
    # '검색일자'를 datetime 객체로 변환
    temp_df['검색일자'] = pd.to_datetime(temp_df['검색일자'])
    
    # 유형에 따라 데이터 필터링
    if '주별' in temp_df['유형'].iloc[0]:
        # 주별 데이터: 7일 간격으로 필터링
        temp_df = temp_df[temp_df.index % 7 == 0]
    elif '월별' in temp_df['유형'].iloc[0]:
        # 월별 데이터: 28일 간격으로 필터링
        temp_df = temp_df[temp_df.index % 28 == 0]
    
    graph_result = pd.concat([graph_result, temp_df], ignore_index=True)

graph_result.head(40)


,검색일자,검색량,연관검색어,유형,오늘일자
0,2020-03-07,0.99262,파킹통장,월별급상승,2024-03-08
1,2020-04-04,20.78460,파킹통장,월별급상승,2024-03-08
2,2020-05-02,1.82987,파킹통장,월별급상승,2024-03-08
3,2020-05-30,2.96491,파킹통장,월별급상승,2024-03-08
4,2020-06-27,1.57092,파킹통장,월별급상승,2024-03-08
5,2020-07-25,2.19239,파킹통장,월별급상승,2024-03-08
6,2020-08-22,1.65724,파킹통장,월별급상승,2024-03-08
7,2020-09-19,1.29472,파킹통장,월별급상승,2024-03-08
8,2020-10-17,1.67882,파킹통장,월별급상승,2024-03-08
9,2020-11-14,1.85145,파킹통장,월별급상승,2024-03-08


# 구글/ 네이버 한꺼번에

In [20]:
import asyncio
import time
from pytrends.request import TrendReq
from concurrent.futures import ThreadPoolExecutor

# 비동기 작업을 위한 Executor 인스턴스를 생성합니다.
executor = ThreadPoolExecutor(max_workers=5)

async def fetch_rising_queries(word: str, max_retries: int = 5) -> dict:
    pytrends = TrendReq(hl='ko-KR', tz=540, retries=2)
    kw_list = [word]

    for attempt in range(max_retries):
        try:
            loop = asyncio.get_running_loop()
            # 비동기로 실행하기 위해 run_in_executor를 사용합니다.
            await loop.run_in_executor(executor, lambda: pytrends.build_payload(kw_list, timeframe='today 1-m', geo='KR', gprop=''))
            result = await loop.run_in_executor(executor, pytrends.related_queries)
            rising_queries = result[word]['rising']
            if rising_queries is not None:
                return {word: list(rising_queries['query'])}
            return {word: []}
        except Exception as e:
            print(f"Error fetching data for {word}: {e}")
            await asyncio.sleep(2 ** attempt)  # Exponential backoff
    return {word: []}  # Return empty dict after max_retries

async def collect_rising_keywords(keywords: list) -> dict:
    tasks = [fetch_rising_queries(word) for word in keywords]
    results = await asyncio.gather(*tasks)
    keyword_results = {}
    for result in results:
        keyword_results.update(result)
    return keyword_results
keywords =(list(info_result_final['연관키워드']))

# 비동기 함수 실행
a = asyncio.run(collect_rising_keywords(keywords))

Error fetching data for 나무증권: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cub098%5Cubb34%5Cuc99d%5Cuad8c%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for ETF투자방법: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22ETF%5Cud22c%5Cuc790%5Cubc29%5Cubc95%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for 퇴직연금수령방법: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-

##### 뉴스링크

In [21]:
import asyncio
import aiohttp
from difflib import SequenceMatcher
import nest_asyncio

nest_asyncio.apply()

def clean_text(text):
    """
    HTML 태그 제거 및 특수 문자 처리 함수
    """
    text = text.replace('<b>', ' ').replace('</b>', ' ')
    text = text.replace('&quot;', '"').replace('&apos;', '\'')
    text = text.replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
    return text

async def fetch_news(session, search_term, headers, params, attempts=10):
    """
    네이버 API를 통해 뉴스 데이터를 비동기적으로 가져오는 함수
    """
    backoff = 1
    for attempt in range(attempts):
        try:
            async with session.get("https://openapi.naver.com/v1/search/news.json", headers=headers, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['items']
                else:
                    raise Exception(f"Error fetching data for {search_term}: {response.status}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < attempts - 1:
                await asyncio.sleep(3 * backoff)  # 실패 시 지수적 백오프 적용
                backoff *= 2
            else:
                raise

async def news_result_async(names):
    """
    검색어별 뉴스 제목, 본문, 링크 추출 및 필터링하는 비동기 함수
    """
    client_id = "ByXmMvAqMIxyVUY_h17L"  # 네이버 API 클라이언트 ID
    client_secret = "2x7yByvNSN"  # 네이버 API 클라이언트 Secret
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_news(session, name, headers, {"query": name, "display": 100, "sort": "date"}) for name in names]
        results = await asyncio.gather(*tasks)
        
        results_dict = {}
        for name, result in zip(names, results):
            titles_links = [(clean_text(item['title']), item['link']) for item in result[:10] if result]
            results_dict[name] = titles_links
            
    return results_dict

async def process_keywords_batch(batch):
    """
    키워드 배치 처리를 위한 함수
    """
    return await news_result_async(batch)

async def main(keywords):
    """
    비동기 실행을 위한 메인 함수, 키워드 리스트를 분할하여 처리
    """
    batch_size = 20 # 한 번에 처리할 키워드 수 # 20 : 160
    batches = [keywords[i:i + batch_size] for i in range(0, len(keywords), batch_size)]
    results = []

    for batch in batches:
        results.append(await process_keywords_batch(batch))
    
    # 결과 병합
    merged_results = {}
    for batch_result in results:
        merged_results.update(batch_result)

    return merged_results

# 실행
news_data = asyncio.run(main(keywords))

for keyword, data in news_data.items():
    print(f"Keyword: {keyword}")
    for title, link in data:
        print(f"Title: {title}, Link: {link}")


Attempt 1 failed: Error fetching data for 인도증시: 429
Attempt 1 failed: Error fetching data for 오늘금시세: 429
Attempt 1 failed: Error fetching data for 암보험: 429
Attempt 1 failed: Error fetching data for 마석역: 429
Attempt 1 failed: Error fetching data for 사망보험: 429
Attempt 1 failed: Error fetching data for 금값: 429
Attempt 1 failed: Error fetching data for 건강보험종류: 429
Attempt 1 failed: Error fetching data for 국제금시세: 429
Attempt 1 failed: Error fetching data for 퇴직연금수령방법: 429
Attempt 1 failed: Error fetching data for 금1돈: 429
Attempt 1 failed: Error fetching data for 오늘의금값: 429
Attempt 1 failed: Error fetching data for 비트코인하는법: 429
Attempt 1 failed: Error fetching data for 실시간국제금시세: 429
Attempt 1 failed: Error fetching data for 금시세1돈: 429
Attempt 1 failed: Error fetching data for 금1돈가격: 429
Attempt 1 failed: Error fetching data for 샌드박스코인: 429
Attempt 1 failed: Error fetching data for 금한돈: 429
Attempt 1 failed: Error fetching data for 금가격: 429
Attempt 1 failed: Error fetching data for 금값시세1돈: 4

In [30]:
# 10개가 아닌 뉴스 항목을 가진 키워드들을 찾습니다.
keys_with_not_ten_items = {key: len(value) for key, value in news_data.items() if len(value) != 10}

keys_with_not_ten_items

{'에쏘팟': 2}

In [60]:

name_list = list(news_data.keys())  
# DataFrame 초기화
a = pd.DataFrame()

# 모든 키워드에 대해 처리
for j in range(len(name_list)):
    # 뉴스 항목이 있는 경우 데이터 추가
    for i in range(len(news_data[name_list[j]])):
        kk = [name_list[j], news_data[name_list[j]][i][0], news_data[name_list[j]][i][1]]
        a = pd.concat([a, pd.DataFrame([kk])], ignore_index=True)

    # 뉴스 항목 수가 10개에 미치지 못하면 나머지를 빈 행으로 채움
    for _ in range(10 - len(news_data[name_list[j]])):
        kk = [name_list[j], None, None]
        a = pd.concat([a, pd.DataFrame([kk])], ignore_index=True)

# 칼럼 이름 설정
a.columns = ['연관키워드', '뉴스제목', '뉴스링크']



In [61]:
a

,연관키워드,뉴스제목,뉴스링크
0,사망보험,"[정인호 객원기자 칼럼] 지금이 의료개혁 절호의 기회, 반드시 살려야",https://weekly.hankooki.com/news/articleView.h...
1,사망보험,"[Health Recipe] 건강 검진, ‘나이’에 맞춤하자",https://n.news.naver.com/mnews/article/057/000...
2,사망보험,"젬퍼리, 진행·재발성 dMMR/MSI-H 자궁내막암 1차 치료 적응증 추가",https://mdtoday.co.kr/news/view/1065585021116355
3,사망보험,"얀센 리브리반트, 폐암 1차 치료 확장…국내 급여 진입 계기 잡나",https://n.news.naver.com/mnews/article/293/000...
4,사망보험,의료공백 속 ‘PA·특례법’ 추진 ‘논란 확대’,http://www.newstomato.com/ReadNews.aspx?no=122...
...,...,...,...
2845,신용카드발급,[금융 단신] KB국민은행·NH농협은행·IBK기업은행 外,http://www.dailypop.kr/news/articleView.html?i...
2846,신용카드발급,12일부터 소상공인·자영업자 ' 신용 사면' 시작된다,http://www.smedaily.co.kr/news/articleView.htm...
2847,신용카드발급,"휴대폰 성지, 말장난 속 갤럭시 S24 '0원'…""정신 안 차리면 당한다"" [르포]",http://www.sisaon.co.kr/news/articleView.html?...
2848,신용카드발급,보이스피싱 수법 갈수록 진화...지속 홍보에도 안 줄어,http://www.hansbiz.co.kr/news/articleView.html...
